In [ ]:
import psycopg2 as ps
from sshtunnel import SSHTunnelForwarder
import configparser as cp
import os

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


## Parse configs

In [ ]:
userpath = os.path.expanduser("~/")
Config = cp.ConfigParser(interpolation=None)
Config.read("{}{}".format(userpath, ".redshift_creds.ini"))

In [ ]:
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [ ]:
try:
    SSH_USERNAME = ConfigSectionMap("REDSHIFT").get('ssh_username')
    DB_USER = ConfigSectionMap("REDSHIFT").get('db_username')
    DB_PASSWORD = ConfigSectionMap("REDSHIFT").get('db_password')
    DB_PORT = ConfigSectionMap("REDSHIFT").get('db_port')
except Exception as e: 
    print(e)
SSH_PRIVATE_KEY = "~/.ssh/id_rsa"
DB_HOST = '127.0.0.1'


## Open tunnel and connect to db

In [ ]:
with SSHTunnelForwarder(
('ec2-107-23-57-72.compute-1.amazonaws.com', 22),
    ssh_username=SSH_USERNAME,
    ssh_private_key=SSH_PRIVATE_KEY,
    remote_bind_address=('localhost', 5439)) as server:

    server.start()
    print("server connected via ssh")
    
    params = {
        'database':'claims',
        'user': DB_USER,
        'password': DB_PASSWORD,
        'host': DB_HOST,
        'port': DB_PORT
    }
    conn = ps.connect(**params)
    curs = conn.cursor()
    print("db connected")


## Get most recent cbrc table name

In [ ]:
most_recent_cbrc_table_name_sql = """SELECT tablename
FROM svv_external_tables
WHERE tablename LIKE 'main_raw_cbrc_bronze%'
ORDER BY tablename DESC
LIMIT 1;"""

curs.execute(most_recent_cbrc_table_name_sql)

table_name = curs.fetchall()[0][0]
table_name = "{}{}".format("claims_spectrum.",table_name)

## Count distinct pids

In [ ]:
distinct_pids = """select count(distinct gr_pid)
from {};""".format(table_name)
curs.execute(distinct_pids)
distinct_pids_res = pd.DataFrame(curs.fetchall(), columns=['distinct pids'])
distinct_pids_res

## Count Distinct NPI's

In [ ]:
distinct_npis = """select count(distinct normalized_npi)
from {};""".format(table_name)
curs.execute(distinct_npis)
distinct_npis_res = pd.DataFrame(curs.fetchall(), columns=['distinct npis'])
distinct_npis_res

## Average Frequency

In [ ]:
average_frequency = """
SELECT AVG(gr_claim_id_count) AS avg_frequency
FROM {}""".format(table_name)
curs.execute(average_frequency)
avg_freq_res = pd.DataFrame(curs.fetchall(), columns=['Average frequency'])
avg_freq_res

## Locations per pid frequency distribution

In [ ]:
# Note that there is a spike on 20 since we limit the number of visits for a patient to 20 
# in the Topk calculation in bronze
locations_per_pid_freq_dist = """
SELECT number_of_visits,
       COUNT(number_of_visits) num_of_visits_count_freq
FROM (
       SELECT gr_pid,
              COUNT(normalized_npi) number_of_visits
       FROM {}
       GROUP BY gr_pid
     ) AS base
GROUP BY number_of_visits
ORDER BY number_of_visits DESC;
""".format(table_name)

curs.execute(locations_per_pid_freq_dist)
loc_per_pid_res = pd.DataFrame(curs.fetchall(), columns=['A. Number of NPI visited', 'Count of pids with A'])
loc_per_pid_res

In [ ]:
sns.barplot(x='A. Number of NPI visited', y='Count of pids with A', data=loc_per_pid_res)

## Records per month

In [ ]:
records_per_month = """
SELECT COUNT(*), date_trunc('mon', visit_end_date) as month
FROM {}
GROUP BY date_trunc('mon', visit_end_date)
ORDER BY month ASC""".format(table_name)

curs.execute(records_per_month)
rec_per_month_res = pd.DataFrame(curs.fetchall(), columns=['Count of visits', 'date'])
rec_per_month_res

## Log Based Histogram to track number of pids per location

In [ ]:
# log based
visits_per_location_hist = """
SELECT Count(b.count_of_count_per_npi) as cc,
       count_of_count_per_npi
FROM
(
  SELECT cast(log(number_of_visits) AS int) count_of_count_per_npi

  FROM (
         SELECT normalized_npi,
                COUNT(gr_pid) number_of_visits
         FROM {}
         GROUP BY normalized_npi
       ) AS base
) as b
GROUP BY count_of_count_per_npi
ORDER BY count_of_count_per_npi;""".format(table_name)

curs.execute(visits_per_location_hist)
visits_per_loc_hist_res = pd.DataFrame(curs.fetchall(), columns=['count', 'log based'])
visits_per_loc_hist_res

## Top 20 visited NPI's

In [ ]:
visits_per_location = """
    SELECT normalized_npi,
        COUNT(gr_pid) number_of_visits
    FROM {}
    GROUP BY normalized_npi
    ORDER BY number_of_visits desc
    limit 20
""".format(table_name)

curs.execute(visits_per_location)
visits_per_location_res = pd.DataFrame(curs.fetchall(), columns=["NPI", "Count"])
visits_per_location_res

## Close Connection

In [ ]:
conn.close()